In [1]:
import os
import pandas as pd
print(pd.__version__)

1.3.5


In [2]:
preprocess_dir = 'storage/preprocessing/220308_baseline/'

In [18]:
def print_memory_usage(obj):
    print('Used '+str(obj.memory_usage(deep=True).sum() / 1024 / 1024 / 1024)+" Gb")
    
def save_csv(df,out_dir,fname):
    df.to_csv(os.path.join(out_dir,fname),index=False)

****Input dataset****

In [4]:
%%time
transactions = pd.read_csv('storage/transactions_train.csv')
articles = pd.read_csv('storage/articles.csv')
customers = pd.read_csv('storage/customers.csv')

CPU times: user 29.7 s, sys: 1.79 s, total: 31.5 s
Wall time: 31.5 s


In [5]:
%%time
from utils import train_test_split
trn_transactions,val_transactions = train_test_split(transactions)
del(transactions)

CPU times: user 4.79 s, sys: 524 ms, total: 5.31 s
Wall time: 5.31 s


In [6]:
trn_transactions = trn_transactions[trn_transactions['t_dat'] > pd.to_datetime('2020-08-01')]

****Feature engineering with past purchase history****

In [ ]:
def past_purchase_count_vector(df,art_df,selected_feature,postfix='_countvec'):
    df = df.merge(art_df[['article_id',selected_feature]],on='article_id')
    df['count'] = 1
    norm = df.groupby(['customer_id'])['count'].count().reset_index()
    norm.rename(columns={'count':'norm'},inplace=True)
    count = df.groupby(['customer_id',selected_feature])['count'].count().reset_index()
    count = count.merge(norm,on='customer_id')
    count['count'] = count['count'] / count['norm']
    count = count.rename(columns={'count':selected_feature+postfix})
    del(norm)
    return count[['customer_id',selected_feature,selected_feature+postfix]]

In [ ]:
%%time
selected_features = [
    'product_group_name', 'product_type_name', 
    'graphical_appearance_name', 'perceived_colour_value_name', 'colour_group_code', 
    'index_name', 'index_group_name', 
    'section_name', 'department_name',
]
for selected_feature in selected_features:
    count = past_purchase_count_vector(trn_transactions[['customer_id','article_id']],articles,selected_feature)
    count.to_csv(preprocess_dir+selected_feature+'_countvec.csv',index=False)

****Feature engineering on repeated purchase****

In [24]:
%%time
trn_transactions['count'] = 1
count = trn_transactions[['customer_id','article_id','count']].groupby(['customer_id','article_id'])['count'].count().reset_index()
norm = trn_transactions[['customer_id','article_id']].groupby('customer_id').count().reset_index().rename(columns={'article_id':'norm'})
count = count.merge(norm,on='customer_id')
count['count'] = count['count'] / count['norm']
count.drop(columns=['norm'],inplace=True)

CPU times: user 2.16 s, sys: 6.12 ms, total: 2.17 s
Wall time: 2.16 s


In [26]:
save_csv(count,preprocess_dir,'repeated_purchase_prob.csv')